# Word2vec preprocessing

Preprocessing is not the most exciting part of NLP, but it is still one of the most important ones. Your task is to preprocess raw text (you can use your own, or [this one](http://mattmahoney.net/dc/text8.zip). For this task text preprocessing mostly consists of:

1. cleaning (mostly, if your dataset is from social media or parsed from the internet)
1. tokenization
1. building the vocabulary and choosing its size. Use only high-frequency words, change all other words to UNK or handle it in your own manner. You can use `collections.Counter` for that.
1. assigning each token a number (numericalization). In other words, make word2index и index2word objects.
1. data structuring and batching - make X and y matrices generator for word2vec (explained in more details below)

**ATTN!:** If you use your own data, please, attach a download link. 

Your goal is to make SkipGramBatcher class which returns two numpy tensors with word indices. It should be possible to use one for word2vec training. You can implement batcher for Skip-Gram or CBOW architecture, the picture below can be helpful to remember the difference.

![text](https://raw.githubusercontent.com/deepmipt/deep-nlp-seminars/651804899d05b96fc72b9474404fab330365ca09/seminar_02/pics/architecture.png)

There are several ways to do it right. Shapes could be `x_batch.shape = (batch_size, 2*window_size)`, `y_batch.shape = (batch_size,)` for CBOW or `(batch_size,)`, `(batch_size,)` for Skip-Gram. You should **not** do negative sampling here.

They should be adequately parametrized: CBOW(window_size, ...), SkipGram(window_size, ...). You should implement only one batcher in this task; and it's up to you which one to chose.

Useful links:
1. [Word2Vec Tutorial - The Skip-Gram Model](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)
1. [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)
1. [Distributed Representations of Words and Phrases and their Compositionality](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)

You can write the code in this notebook, or in a separate file. It can be reused for the next task. The result of your work should represent that your batch has a proper structure (right shapes) and content (words should be from one context, not some random indices). To show that, translate indices back to words and print them to show something like this:

```
text = ['first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including']

window_size = 2

# CBOW:
indices_to_words(x_batch) = \
        [['first', 'used', 'early', 'working'],
        ['used', 'against', 'working', 'class'],
        ['against', 'early', 'class', 'radicals'],
        ['early', 'working', 'radicals', 'including']]

indices_to_words(labels_batch) = ['against', 'early', 'working', 'class']

# Skip-Gram

indices_to_words(x_batch) = ['against', 'early', 'working', 'class']

indices_to_words(labels_batch) = ['used', 'working', 'early', 'radicals']]

```

If you struggle with something, ask your neighbor. If it is not obvious for you, probably someone else is looking for the answer too. And in contrast, if you see that you can help someone - do it! Good luck!

In [1]:
from pathlib import Path
from collections import Counter
from itertools import islice, product, chain

import numpy as np

In [2]:
DATA_PATH = Path('../data')

There is an implementation of SkipGramBatch (for text8 dataset) with the following steps:
- Load text from file  
- Tokenize text by whitespace  
- Count tokens' frequencies  
- Build vocabulary  
    - Filter tokens by frequency threshold (cutoff)
    - Build 2 mappings: token to indices and indices to tokens
    - Build vocabulary as filtered set of tokens
- Filter tokens in text by frequency (with the same parameters as for vocabulary)
    - Tokens with frequency less than cutoff are removed
- Vectorize filtered tokens
- Create sliding window with specific window size (window_size)
    - For tokens at the beginning and end of the text there are windows with smaller sizes, no padding is used
- Generate batches based on specific batch size (batch_size)
    - Whether to use last batch, if its length is smaller than batch size, is controlled by parameter drop_last (default value: True)

Notes:
- There is only one line in original dataset, so the full size of original text is read in RAM, and there are no additional tokens are used (e.g. for marking sentences' boundaries)
- This implementation is based on generating batches by Python generators, so shuffling is not used. Perhaps it is not critical here because we will have tens of millions of input/output pairs

In [653]:
class SkipGramBatcher():
    def __init__(self, text):
        self.text = text
    
    @classmethod
    def from_file(cls, file_path):
        with open(file_path) as f:
            text = f.read()
        
        return cls(text)
    
    def _tokenize(self):
        self.tokens = self.text.split()
    
    def _count_tokens(self):
        self.token_counts = Counter(self.tokens)
    
    def _build_vocab(self, cutoff):
        filtered_token_counts = dict(filter(lambda x: x[1] >= cutoff, self.token_counts.items()))
        self.token_to_idx = {token:idx for (idx, (token, _)) 
                             in enumerate(filtered_token_counts.items())}
        self.idx_to_token = {idx:token for (token, idx) in self.token_to_idx.items()}
        self.vocab = set(self.token_to_idx)

    def _filter_tokens(self):
        self.tokens = [token for token in self.tokens if token in self.vocab]
    
    def _vectorize_tokens(self):
        self.vectorized_tokens = [self.token_to_idx[token] for token in self.tokens]
    
    def _create_sliding_window(self, window_size):
        tokens_size = len(self.tokens)

        for i in range(0, tokens_size):
            center_word = self.vectorized_tokens[i:i+1]
            left_context = self.vectorized_tokens[max(0, i - window_size): i]
            right_context = self.vectorized_tokens[i + 1: min(tokens_size, i + window_size + 1)]
            context = left_context + right_context
            window = [list(product(center_word, context))]
            yield window 
    
    def devectorize_tokens(self, indices):
        return [self.idx_to_token[idx] for idx in indices]
        
    def prepare_data(self, cutoff=1):
        self._tokenize()
        self._count_tokens()
        self._build_vocab(cutoff)
        self._filter_tokens()
        self._vectorize_tokens()
        
    def generate_batches(self, window_size=1, batch_size=1, drop_last=True):
        window = self._create_sliding_window(window_size)
        batch = list(zip(*islice(window, batch_size)))
        
        if drop_last:
            while batch and len(batch[0]) == batch_size:
                batch = list(zip(*[pair for pairs in batch[0] for pair in pairs]))
                yield batch
                batch = list(zip(*islice(window, batch_size)))
        else:
            while batch:
                batch = list(zip(*[pair for pairs in batch[0] for pair in pairs]))
                yield batch
                batch = list(zip(*islice(window, batch_size)))

In [654]:
sg_batcher = SkipGramBatcher.from_file(DATA_PATH/'text8')
sg_batcher.prepare_data(cutoff=100000)

In [655]:
len(sg_batcher.tokens)

5656412

In [656]:
list(chain([[[1, 2], [3, 4]]]))

[[[1, 2], [3, 4]]]

In [660]:
g = sg_batcher.generate_batches(window_size=3, batch_size=1024)

In [483]:
arr = np.array([list([(0, 1), (0, 2)]), list([(1, 0), (1, 2), (1, 3)]),
       list([(2, 0), (2, 1), (2, 3), (2, 4)])])
list(zip(*[pair for pairs in arr for pair in pairs]))

[(0, 0, 1, 1, 1, 2, 2, 2, 2), (1, 2, 0, 2, 3, 0, 1, 3, 4)]

In [238]:
type(arr[0][0][0])

tuple

In [120]:
arr = [([(0, 1), (0, 2)], [(1, 0), (1, 2), (1, 3)], [(2, 0), (2, 1), (2, 3), (2, 4)])]
arr

[([(0, 1), (0, 2)],
  [(1, 0), (1, 2), (1, 3)],
  [(2, 0), (2, 1), (2, 3), (2, 4)])]

In [133]:
list(map(lambda x: list(zip(*x)), arr[0]))

[[(0, 0), (1, 2)], [(1, 1, 1), (0, 2, 3)], [(2, 2, 2, 2), (0, 1, 3, 4)]]

In [141]:
list(zip(*[pair for pairs in arr[0] for pair in pairs]))

[(0, 0, 1, 1, 1, 2, 2, 2, 2), (1, 2, 0, 2, 3, 0, 1, 3, 4)]

For testing, let's try to create batcher, generate batch and check correctness of batch's content

In [651]:
cutoff = 10
window_size = 3
batch_size = 10

sg_batcher = SkipGramBatcher.from_file(DATA_PATH/'text8')
sg_batcher.prepare_data(cutoff=cutoff)

g = sg_batcher.generate_batches(window_size=window_size, batch_size=batch_size)
x_batch, labels_batch = next(g)

x_tokens_batch = sg_batcher.devectorize_tokens(x_batch)
labels_tokens_batch = sg_batcher.devectorize_tokens(labels_batch)

print('\nFirst original words in text: ')
print(' '.join(sg_batcher.text.split()[:batch_size]))

print('\nFirst pre-processed tokens:')
print(sg_batcher.tokens[:batch_size])

print('\nFirst vectorized tokens:')
print(sg_batcher.vectorized_tokens[:batch_size])

print('\nCutoff:')
print(cutoff)

print('\nWindow size:')
print(window_size)

print('\nBatch size:')
print(batch_size)

print('\nFirst token-index mappings:')
print(list(sg_batcher.token_to_idx.items())[:batch_size])

print('\nVocabulary size:')
print(len(sg_batcher.vocab))

print('\nBatch x indices:')
print(repr(x_batch))

print('\nBatch x tokens:')
print(x_tokens_batch)

print('\nBatch labels indices:')
print(repr(labels_batch))

print('\nBatch labels tokens:')
print(labels_tokens_batch)

print('\nBatch pairs indices:')
print(list(zip(x_batch, labels_batch)))

print('\nBatch pairs tokens:')
print(list(zip(x_tokens_batch, labels_tokens_batch)))

10

First original words in text: 
anarchism originated as a term of abuse first used against

First pre-processed tokens:
['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against']

First vectorized tokens:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

Cutoff:
10

Window size:
3

Batch size:
10

First token-index mappings:
[('anarchism', 0), ('originated', 1), ('as', 2), ('a', 3), ('term', 4), ('of', 5), ('abuse', 6), ('first', 7), ('used', 8), ('against', 9)]

Vocabulary size:
47134

Batch x indices:
(0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9)

Batch x tokens:
['anarchism', 'anarchism', 'anarchism', 'originated', 'originated', 'originated', 'originated', 'as', 'as', 'as', 'as', 'as', 'a', 'a', 'a', 'a', 'a', 'a', 'term', 'term', 'term', 'term', 'term', 'term', 'of', 'of', 'of', 'of', 'of', 'of', 'abuse', 'abuse', 'abuse', 'abuse', 'abuse', 'abuse', '